In [1]:
from sklearn.datasets import load_files       
from keras.utils import np_utils
import numpy as np
from glob import glob
import cv2
import random
from tqdm import tqdm

#main_folder = "C:/Users/kvvip/Udacity_MLND/Capstone/tennis/tennis_vedios/"
#main_folder = "/media/karthik/Windows/Users/kvvip/Udacity_MLND/Capstone/Capstone_Udacity/"
main_folder = "" #main folder is current folder

def load_datasets(path):
    data = load_files(path)
    tennis_files = data['filenames']
    tennis_targets = np_utils.to_categorical(np.array(data['target']), 3)
    return tennis_files, tennis_targets

train_files, train_targets = load_datasets(main_folder+"train_yolo/")
valid_files, valid_targets = load_datasets(main_folder + "valid_yolo/")
test_files, test_targets = load_datasets(main_folder + "test_yolo/")

#print(test_targets)

Using TensorFlow backend.


In [2]:


def video_to_images(num, video_path, out_shape):
    # divide the video file into "num" equal parts and randomly sample one
    # image from each part, thereby, totalling "num" images per video file
    
    video_reader = cv2.VideoCapture(video_path)

    nb_frames = int(video_reader.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_h = int(video_reader.get(cv2.CAP_PROP_FRAME_HEIGHT))
    frame_w = int(video_reader.get(cv2.CAP_PROP_FRAME_WIDTH))
    #print(nb_frames, frame_h, frame_w, video_path)
    
    frame_indexs= list(np.array(range(0,nb_frames+1, round(nb_frames/(num)))))
    #np.append(frame_indexs,nb_frames)
    #print(frame_indexs)
    
    if len(frame_indexs) != (num + 1):
        frame_indexs.append(nb_frames)
    
    frame_indexs = np.array(frame_indexs)
    
    index_start=0
    selected_frames = []
    images_out=np.array([])
    #print(frame_indexs)
    for vals in frame_indexs[1:]:
        selected_frames.append(random.choice(range(index_start, vals)))
        index_start = vals
        
        video_reader.set(1, selected_frames[-1])
        ret, image = video_reader.read()
        
        #print(image.shape)
        input_image = cv2.resize(image, out_shape)
        #print(input_image.shape)
        input_image = input_image/ 255 #normalizing
        #input_image=list(input_image)
        try:
            images_out = np.append(images_out,np.expand_dims(input_image, axis=0), axis=0)
            #print(images_out.shape, "here")
        except:
            images_out= np.array(np.expand_dims(input_image, axis=0))
            #print("except", images_out.shape)
            continue
    
    
    #images_out  = np.expand_dims(images_out, axis=0)
    video_reader.release()
    
    #print(images_out.shape)
    
    return images_out
        
    #print(selected_frames)
    
    
    
out_images = video_to_images(5, train_files[0], (640, 360))
print(out_images.shape, np.expand_dims(out_images, axis=0).shape)

(5, 360, 640, 3) (1, 5, 360, 640, 3)


In [3]:
#creating train, valid and test tensors

def get_tensors(files):
    
    tensors_out = video_to_images(5, files[0] , (299,299))
    
    for path_ in tqdm(files[1:]):
        tensors_out = np.vstack((tensors_out, video_to_images(5, path_ , (299,299))))
    
    return tensors_out

#train_tensors = get_tensors(train_files)
#valid_tensors = get_tensors(valid_files)
#test_tensors = get_tensors(test_files)
        
train_tensors = np.array([video_to_images(5, path_ , (299,299)) for path_ in tqdm(train_files)])
valid_tensors = np.array([video_to_images(5, path_ , (299,299)) for path_ in tqdm(valid_files)])
test_tensors = np.array([video_to_images(5, path_ , (299,299)) for path_ in tqdm(test_files)])


100%|██████████| 101/101 [00:01<00:00, 50.78it/s]


In [4]:
print(train_tensors.shape, type(train_tensors))
print(valid_tensors.shape, type(valid_tensors))
print(test_tensors.shape, type(test_tensors))

(300, 5, 299, 299, 3) <class 'numpy.ndarray'>
(100, 5, 299, 299, 3) <class 'numpy.ndarray'>
(101, 5, 299, 299, 3) <class 'numpy.ndarray'>


In [5]:
from keras.layers import Conv3D, MaxPooling3D, GlobalAveragePooling3D,MaxPooling2D, Conv2D
from keras.layers import Dropout, Flatten, Dense
from keras.layers import Input, AveragePooling3D, GlobalAveragePooling2D
from keras.models import Sequential
from keras.layers.merge import concatenate



model = Sequential()
model.add(Conv3D(8, (2,2,2), strides=(1,2,2), padding='same', activation='relu', input_shape=(5, 299, 299, 3)))
model.add(MaxPooling3D(pool_size=(1,2,2), strides=None, padding='valid', data_format="channels_last"))
#model.add(Dropout(0.1))

model.add(Conv3D(16, (2,2,2), strides=(1,2,2), padding='same', activation='relu'))
model.add(MaxPooling3D(pool_size=(1,2,2), strides=None, padding='valid', data_format="channels_last"))
#model.add(Dropout(0.1))

model.add(Conv3D(32, (2,2,2), strides=(1,2,2), padding='same', activation='relu'))
model.add(MaxPooling3D(pool_size=(2,2,2), strides=None, padding='valid', data_format="channels_last"))
#model.add(Dropout(0.1))

model.add(Conv3D(64, (2,2,2), strides=(1,2,2), padding='same', activation='relu'))
model.add(MaxPooling3D(pool_size=(2,2,2), strides=None, padding='valid', data_format="channels_last"))


model.add(Flatten())
model.add(Dense(3, activation='softmax'))

model.summary()




_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_1 (Conv3D)            (None, 5, 150, 150, 8)    200       
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 5, 75, 75, 8)      0         
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 5, 38, 38, 16)     1040      
_________________________________________________________________
max_pooling3d_2 (MaxPooling3 (None, 5, 19, 19, 16)     0         
_________________________________________________________________
conv3d_3 (Conv3D)            (None, 5, 10, 10, 32)     4128      
_________________________________________________________________
max_pooling3d_3 (MaxPooling3 (None, 2, 5, 5, 32)       0         
_________________________________________________________________
conv3d_4 (Conv3D)            (None, 2, 3, 3, 64)       16448     
__________

In [6]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [7]:
from keras.callbacks import ModelCheckpoint  

### specify the number of epochs that you would like to use to train the model.

epochs = 20


checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.from_scratch_yolo.hdf5', 
                               verbose=1, save_best_only=True)

model.fit(train_tensors, train_targets, 
          validation_data=(valid_tensors, valid_targets),
          epochs=epochs, batch_size=20, callbacks=[checkpointer], verbose=2)

Train on 300 samples, validate on 100 samples
Epoch 1/20
Epoch 00001: val_loss improved from inf to 1.07053, saving model to saved_models/weights.best.from_scratch_yolo.hdf5
 - 3s - loss: 1.0877 - acc: 0.4067 - val_loss: 1.0705 - val_acc: 0.4200
Epoch 2/20
Epoch 00002: val_loss improved from 1.07053 to 1.06533, saving model to saved_models/weights.best.from_scratch_yolo.hdf5
 - 2s - loss: 1.0700 - acc: 0.4200 - val_loss: 1.0653 - val_acc: 0.4200
Epoch 3/20
Epoch 00003: val_loss improved from 1.06533 to 1.05536, saving model to saved_models/weights.best.from_scratch_yolo.hdf5
 - 2s - loss: 1.0609 - acc: 0.4500 - val_loss: 1.0554 - val_acc: 0.4200
Epoch 4/20
Epoch 00004: val_loss did not improve
 - 2s - loss: 1.0545 - acc: 0.4333 - val_loss: 1.0581 - val_acc: 0.4200
Epoch 5/20
Epoch 00005: val_loss improved from 1.05536 to 1.02224, saving model to saved_models/weights.best.from_scratch_yolo.hdf5
 - 2s - loss: 1.0352 - acc: 0.4200 - val_loss: 1.0222 - val_acc: 0.4200
Epoch 6/20
Epoch 0000

In [8]:
model.load_weights('saved_models/weights.best.from_scratch_yolo.hdf5')

In [9]:
# get index of predicted dog breed for each image in test set
tennis_predictions = [np.argmax(model.predict(np.expand_dims(tensor, axis=0))) for tensor in test_tensors]

# report test accuracy
test_accuracy = 100*np.sum(np.array(tennis_predictions)==np.argmax(test_targets, axis=1))/len(tennis_predictions)
print('Test accuracy: %.4f%%' % test_accuracy)

Test accuracy: 76.2376%


**Applying InceptionV3 transfer learning and using Conv3d on the reultant tensors.**

In [10]:
from keras.applications.inception_v3 import InceptionV3

input_tensor = Input(shape=(299, 299, 3))
# create the base pre-trained model
base_model = InceptionV3(input_tensor=input_tensor, weights='imagenet', include_top=False)


In [11]:
base_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 299, 299, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 149, 149, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 149, 149, 32) 96          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 149, 149, 32) 0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

In [12]:
def bottleneck_feature_extractor(inp_tensors, model_used, length=300, num_images=5):
    
    #train tensors are of shape (300, 5, 299,299,3)

    copy_inp_tensors= np.reshape(inp_tensors, (num_images, length, 299, 299, 3))
    #copy_inp_tensors of shape (5, 300, 299,299,3)
    bottleneck_features=[]
    for frame_images in tqdm(copy_inp_tensors):
        bottleneck_features.append(model_used.predict(frame_images))
        
    #BNeck_tensors1 = np.squeeze(np.split(np.array(bottleneck_features), length, axis=1))
    bottleneck_features = np.array(bottleneck_features)
    a,b,c,d,e = bottleneck_features.shape
    BNeck_tensors = np.reshape(bottleneck_features, (b, a, c, d, e))
    #print(BNeck_tensors == BNeck_tensors1)
    return BNeck_tensors


train_BNeck_tensors = bottleneck_feature_extractor(train_tensors, base_model)
valid_BNeck_tensors = bottleneck_feature_extractor(valid_tensors, base_model, len(valid_tensors),5)
test_BNeck_tensors = bottleneck_feature_extractor(test_tensors, base_model, len(test_tensors), 5)

print(train_BNeck_tensors.shape, valid_BNeck_tensors.shape, test_BNeck_tensors.shape)
   
        
   
        
    
    

100%|██████████| 5/5 [00:04<00:00,  1.20it/s]

(300, 5, 8, 8, 2048) (100, 5, 8, 8, 2048) (101, 5, 8, 8, 2048)


In [13]:
BNeck_model = Sequential()
BNeck_model.add(Conv3D(32, (2,2,2), strides=(2,2,2), padding='same', activation='relu', input_shape=(5, 8, 8, 2048), data_format="channels_first"))
BNeck_model.add(MaxPooling3D(pool_size=(2,2,2), strides=None, padding='valid', data_format="channels_first"))
model.add(Dropout(0.8))

BNeck_model.add(Conv3D(64, (2,2,2), strides=(2,2,2), padding='same', activation='relu', data_format="channels_first"))
#BNeck_model.add(MaxPooling3D(pool_size=(1,2,2), strides=None, padding='valid', data_format="channels_first"))
model.add(Dropout(0.8))

#BNeck_model.add(Conv3D(32, (2,2,2), strides=(2,2,2), padding='same', activation='relu'))
#BNeck_model.add(MaxPooling3D(pool_size=(2,2,2), strides=None, padding='valid', data_format="channels_last"))
#model.add(Dropout(0.1))


BNeck_model.add(Flatten())
#BNeck_model.add(Dense(16, activation='relu'))
#model.add(Dropout(0.8))
BNeck_model.add(Dense(3, activation='softmax'))

BNeck_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_5 (Conv3D)            (None, 32, 4, 4, 1024)    1312      
_________________________________________________________________
max_pooling3d_5 (MaxPooling3 (None, 32, 2, 2, 512)     0         
_________________________________________________________________
conv3d_6 (Conv3D)            (None, 64, 1, 1, 256)     16448     
_________________________________________________________________
flatten_2 (Flatten)          (None, 16384)             0         
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 49155     
Total params: 66,915
Trainable params: 66,915
Non-trainable params: 0
_________________________________________________________________


In [14]:
BNeck_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


### specify the number of epochs that you would like to use to train the model.

epochs = 40


checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.from_InceptionV3_yolo.hdf5', 
                               verbose=1, save_best_only=True)

BNeck_model.fit(train_BNeck_tensors, train_targets, 
          validation_data=(valid_BNeck_tensors, valid_targets),
          epochs=epochs, batch_size=20, callbacks=[checkpointer], verbose=2)

Train on 300 samples, validate on 100 samples
Epoch 1/40
Epoch 00001: val_loss improved from inf to 0.98139, saving model to saved_models/weights.best.from_InceptionV3_yolo.hdf5
 - 1s - loss: 1.2405 - acc: 0.4467 - val_loss: 0.9814 - val_acc: 0.4100
Epoch 2/40
Epoch 00002: val_loss improved from 0.98139 to 0.58812, saving model to saved_models/weights.best.from_InceptionV3_yolo.hdf5
 - 1s - loss: 0.6770 - acc: 0.7700 - val_loss: 0.5881 - val_acc: 0.8000
Epoch 3/40
Epoch 00003: val_loss improved from 0.58812 to 0.35737, saving model to saved_models/weights.best.from_InceptionV3_yolo.hdf5
 - 1s - loss: 0.2999 - acc: 0.9367 - val_loss: 0.3574 - val_acc: 0.8800
Epoch 4/40
Epoch 00004: val_loss did not improve
 - 1s - loss: 0.1195 - acc: 0.9700 - val_loss: 0.3962 - val_acc: 0.8500
Epoch 5/40
Epoch 00005: val_loss improved from 0.35737 to 0.18929, saving model to saved_models/weights.best.from_InceptionV3_yolo.hdf5
 - 1s - loss: 0.0734 - acc: 0.9900 - val_loss: 0.1893 - val_acc: 0.9300
Epoch

In [15]:
# get index of predicted dog breed for each image in test set
tennis_predictions = [np.argmax(BNeck_model.predict(np.expand_dims(tensor, axis=0))) for tensor in test_BNeck_tensors]

# report test accuracy
test_accuracy = 100*np.sum(np.array(tennis_predictions)==np.argmax(test_targets, axis=1))/len(tennis_predictions)
print('Test accuracy: %.4f%%' % test_accuracy)

Test accuracy: 91.0891%


**Applying InceptionV3 transfer learning and using Concatenation+Conv2D model on the reultant tensors.**

In [16]:
from keras.layers.merge import Concatenate

def bottleneck_feature_extractor(inp_tensors, model_used, length=300, num_images=5):
    
    #train tensors are of shape (300, 5, 299,299,3)
    copy_inp_tensors= np.reshape(inp_tensors, (num_images, length, 299, 299, 3))
    #copy_inp_tensors of shape (5, 300, 299,299,3)
    bottleneck_features=[]
    #bottleneck_features = model_used.predict(copy_inp_tensors[0])
    for frame_images in tqdm(copy_inp_tensors[:]):
        bottleneck_features.append(model_used.predict(frame_images))
        #bottleneck_features = Concatenate([bottleneck_features, model_used.predict(frame_images)])
        
    #BNeck_tensors1 = np.squeeze(np.split(np.array(bottleneck_features), length, axis=1))
    bottleneck_features = np.array(bottleneck_features)
    #print(bottleneck_features.shape)
    a,b,c,d,e = bottleneck_features.shape
    BNeck_tensors = np.reshape(bottleneck_features, (b, a, c, d, e))
    BNeck_tensors = np.reshape(bottleneck_features, (b, a*c, d, e))
    #print(BNeck_tensors == BNeck_tensors1)
    #BNeck_tensors = bottleneck_features
    return BNeck_tensors


train_BNeck_tensors = bottleneck_feature_extractor(train_tensors, base_model)
valid_BNeck_tensors = bottleneck_feature_extractor(valid_tensors, base_model, len(valid_tensors),5)
test_BNeck_tensors = bottleneck_feature_extractor(test_tensors, base_model, len(test_tensors), 5)

print(train_BNeck_tensors.shape, valid_BNeck_tensors.shape, test_BNeck_tensors.shape)

100%|██████████| 5/5 [00:04<00:00,  1.24it/s]

(300, 40, 8, 2048) (100, 40, 8, 2048) (101, 40, 8, 2048)


In [17]:
BNeck_model = Sequential()
BNeck_model.add(Conv2D(32, (4,2), strides=(1,1), padding='same', activation='relu', input_shape=(40, 8, 2048)))#, data_format="channels_first"))
BNeck_model.add(MaxPooling2D(pool_size=(2,2), strides=None, padding='valid'))#, data_format="channels_first"))
model.add(Dropout(0.8))

BNeck_model.add(Conv2D(64, (4,2), strides=(2,2), padding='same', activation='relu'))#, data_format="channels_first"))
#BNeck_model.add(MaxPooling3D(pool_size=(1,2,2), strides=None, padding='valid', data_format="channels_first"))
model.add(Dropout(0.8))



BNeck_model.add(Flatten())
#BNeck_model.add(Dense(16, activation='relu'))
#model.add(Dropout(0.8))
BNeck_model.add(Dense(3, activation='softmax'))

BNeck_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_95 (Conv2D)           (None, 40, 8, 32)         524320    
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 20, 4, 32)         0         
_________________________________________________________________
conv2d_96 (Conv2D)           (None, 10, 2, 64)         16448     
_________________________________________________________________
flatten_3 (Flatten)          (None, 1280)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 3843      
Total params: 544,611
Trainable params: 544,611
Non-trainable params: 0
_________________________________________________________________


In [18]:
BNeck_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


### specify the number of epochs that you would like to use to train the model.

epochs = 40


checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.from_InceptionV3.hdf5', 
                               verbose=1, save_best_only=True)

BNeck_model.fit(train_BNeck_tensors, train_targets, 
          validation_data=(valid_BNeck_tensors, valid_targets),
          epochs=epochs, batch_size=20, callbacks=[checkpointer], verbose=2)

Train on 300 samples, validate on 100 samples
Epoch 1/40
Epoch 00001: val_loss improved from inf to 0.54490, saving model to saved_models/weights.best.from_InceptionV3.hdf5
 - 1s - loss: 1.1456 - acc: 0.5500 - val_loss: 0.5449 - val_acc: 0.7900
Epoch 2/40
Epoch 00002: val_loss improved from 0.54490 to 0.49974, saving model to saved_models/weights.best.from_InceptionV3.hdf5
 - 0s - loss: 0.2670 - acc: 0.9067 - val_loss: 0.4997 - val_acc: 0.7800
Epoch 3/40
Epoch 00003: val_loss improved from 0.49974 to 0.25273, saving model to saved_models/weights.best.from_InceptionV3.hdf5
 - 1s - loss: 0.0661 - acc: 0.9800 - val_loss: 0.2527 - val_acc: 0.9000
Epoch 4/40
Epoch 00004: val_loss improved from 0.25273 to 0.18194, saving model to saved_models/weights.best.from_InceptionV3.hdf5
 - 0s - loss: 0.0147 - acc: 1.0000 - val_loss: 0.1819 - val_acc: 0.9300
Epoch 5/40
Epoch 00005: val_loss improved from 0.18194 to 0.17052, saving model to saved_models/weights.best.from_InceptionV3.hdf5
 - 0s - loss: 0

In [19]:
# get index of predicted dog breed for each image in test set
tennis_predictions = [np.argmax(BNeck_model.predict(np.expand_dims(tensor, axis=0))) for tensor in test_BNeck_tensors]

# report test accuracy
test_accuracy = 100*np.sum(np.array(tennis_predictions)==np.argmax(test_targets, axis=1))/len(tennis_predictions)
print('Test accuracy: %.4f%%' % test_accuracy)

Test accuracy: 92.0792%
